# Preparation

This part will help you get access to Gemini API and run your first prompting task.

For a detailed API walkthrough, see [this documentation page](https://ai.google.dev/tutorials/python_quickstart).


In [2]:
import sys
import os
import logging
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

/Users/eason/anaconda3/envs/py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Setup your API key
Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in [Google AI Studio](https://makersuite.google.com/app/apikey). After obtaining the API key add it to your environment variables.

In [3]:
import dotenv
dotenv.load_dotenv()
GOOGLE_API_KEY=os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## List models

Now you're ready to call the Gemini API. Use list_models to see the available Gemini models:


In [4]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


## Use this part to do your prompting

use the `gemini-pro` model.

In [5]:
model = genai.GenerativeModel('gemini-pro')
response = model.generate_content("What is HCI?")
to_markdown(response.text)

> Human-Computer Interaction (HCI)

# Writeup 0: Task and Strategy Description

Here, you should describe what task you are working on, and what workflow/pipeline you intend to replicate (from which crowdsourcing paper). As a reminder, you should [pick a crowdsourcing paper here](https://docs.google.com/spreadsheets/d/1qM66oZ5YcSjqifegoyNgl3LFMaCfFHgoPe8isgckbXA/edit?usp=sharing). The spreadsheet also points to example tasks in the crowdsourcing papers; However, you DON'T have to stick to the paper-provided input/output. Please feel free to come up with your own tasks as long as they seem suitable for the paper/pipeline you are replicating.
 

**EDIT THIS PART TO PROVIDE AN OVERVIEW OF YOUR ATTEMPTS**

- **Task Description**: Sort a list of array by the number of odd number in the array element
- **Example Input/output**: Write >=3 input-output pairs of your task. You should test your strategy on all the three examples.
    ```
    Input: [[1,3,5,7,9], [2,4,6,8,10], [1,2,3,4,5], [1,1,5,7,9], [1,5,5,7,9]]
    Output: [[2, 4, 6, 8, 10], [1, 2, 3, 4, 5], [1, 3, 5, 7, 9], [1, 1, 5, 7, 9], [1, 5, 5, 7, 9]]
    
    Input: [[58, 37, 46, 77, 47], [47, 79, 44, 9, 30], [54, 26, 88, 74, 10], [9, 20, 80, 63, 41], [41, 35, 80, 94, 15]]
    Output: [[54, 26, 88, 74, 10], [41, 35, 80, 94, 15], [9, 20, 80, 63, 41], [47, 79, 44, 9, 30], [58, 37, 46, 77, 47]]
    
    Input: [[90, 54, 48, 36, 89], [95, 59, 64, 20, 87], [40, 28, 52, 48, 94], [73, 72, 38, 70, 47], [19, 76, 61, 68, 9]]
    Output: [[40, 28, 52, 48, 94], [90, 54, 48, 36, 89], [73, 72, 38, 70, 47], [95, 59, 64, 20, 87], [19, 76, 61, 68, 9]]
    ```
- **Workflow prompting strategy**: 
    1. Assign an index to each list in JSON format, key is index, value is the list.
     2. Identify each number in the list is odd or even in a JSON format, for example, if the input lists is [[1, 2, 3, 4, 5], [6, 7, 8, 9, 10]], the output should be {{ "0": {{"1": "odd", "2": "even", "3": "odd", "4": "even", "5": "odd"}}, "1": {{"6": "even", "7": "odd", "8": "even", "9": "odd", "10": "even"}} }}.
     3. grab all odd numbers are in each list in the json format, key is index, value is all odd numbers. Remember, odd numbers ends with 1, 3, 5, 7, 9. You should not include any even numbers that ends with 2, 4, 6, 8, 0. For example, 81, 73, 55, 37, 19 are odd number, while 82, 74, 56, 38, 20 are even numbers.
     4. Indicated how many odd numbers are in each list in the json format, key is index, value is how many odd numbers it includes.
     5. Sort the lists in order of how many odd numbers they contain. Output a json list, each element is the index of the list, sorted in order of how many odd numbers they contain.
     6. Sort the lists in order of how many odd numbers they contain. Output a json list, each element is the list input, sorted in order of how many odd numbers they contain."""
    
- **Crowdsourcing paper**: [Cheng, Justin, et al. "Break it down: A comparison of macro-and microtasks." CHI 2015.](https://hci.stanford.edu/publications/2015/micromacro/micromacro.pdf) -- where the pipelining strategy comes from.

# Baseline Prompting for Sorting

In [72]:
# Write your code here.
# Write your code here.
def baseline_prompt(random_lists, model='gemini-pro'):
    prompt = f"""I have some random lists of numbers: {random_lists}. Please sort it in order of how many odd numbers they contain."""
    
    model = genai.GenerativeModel('gemini-pro')
    response = model.generate_content(prompt)
    print(f"INPUT: {random_lists}")
    print(f"OUTPUT: {response.text}")
    print()


# Prepare the list useing random
import random
random_lists = [random.sample(range(1, 100), 5) for _ in range(5)]
baseline_prompt(random_lists)
print("We can see the result is not very good... It is not accurate and not useful. Let's try to improve it by chain of thought.")

INPUT: [[90, 54, 48, 36, 89], [95, 59, 64, 20, 87], [40, 28, 52, 48, 94], [73, 72, 38, 70, 47], [19, 76, 61, 68, 9]]
OUTPUT: 1. [19, 76, 61, 68, 9] (2 odd numbers)
2. [40, 28, 52, 48, 94] (2 odd numbers)
3. [90, 54, 48, 36, 89] (2 odd numbers)
4. [95, 59, 64, 20, 87] (1 odd number)
5. [73, 72, 38, 70, 47] (1 odd number)

We can see the result is not very good... It is not accurate and not useful. Let's try to improve it by chain of thought.


# Advanced Prompting for sorting with chain of thought
We can see the result is nearly perfect.

I make several modification of this prompt, at first I just ask gemini to sort it and output. Then I found that it need to use index as an indicator at (1) to lower the length and prevent hallucination. Then, I found it still can't catch all odd numbers, so I ask gemini to double check at (3). Then, the result is perfect! Oh Yes!

In [39]:
# Write your code here.
# Write your code here.
def advanced_prompt(random_lists, model='gemini-pro'):
    prompt = f"""I have some random lists of numbers: {random_lists}. 
    Please do the following using text only, don't output any code:
     1. Assign an index to each list in JSON format, key is index, value is the list.
     2. grab all odd numbers are in each list in the json format, key is index, value is all odd numbers. Remember, odd numbers ends with 1, 3, 5, 7, 9. You should not include any even numbers that ends with 2, 4, 6, 8, 0. For example, 81, 73, 55, 37, 19 are odd number, while 82, 74, 56, 38, 20 are even numbers.
     3. double check and output all odd numbers are in each list in the json format again, if you miss any odd numbers in the list, please add it. 
     3. Indicated how many odd numbers are in each list in the json format, key is index, value is how many odd numbers it includes.
     4. Sort the lists in order of how many odd numbers they contain. Output a json list, each element is the index of the list, sorted in order of how many odd numbers they contain.
     5. Sort the lists in order of how many odd numbers they contain. Output a json list, each element is the list input, sorted in order of how many odd numbers they contain."""
    
    model = genai.GenerativeModel('gemini-pro')
    response = model.generate_content(prompt)
    print(f"INPUT: {random_lists}")
    print(f"OUTPUT: {response.text}")
    print()


# Prepare the list useing random
import random
random_lists = [random.sample(range(1, 100), 5) for _ in range(5)]
advanced_prompt(random_lists)

INPUT: [[73, 19, 40, 86, 90], [91, 23, 25, 54, 26], [67, 5, 40, 24, 44], [82, 58, 31, 42, 73], [31, 55, 5, 91, 63]]
OUTPUT: **1. Assign an index to each list in JSON format:**
```
{
  "0": [73, 19, 40, 86, 90],
  "1": [91, 23, 25, 54, 26],
  "2": [67, 5, 40, 24, 44],
  "3": [82, 58, 31, 42, 73],
  "4": [31, 55, 5, 91, 63]
}
```

**2. Grab all odd numbers in each list in JSON format:**
```
{
  "0": [19, 90],
  "1": [23, 25, 55],
  "2": [],
  "3": [31, 73],
  "4": [31, 55, 91, 63]
}
```

**3. Double check and output all odd numbers in each list in JSON format again:**
```
{
  "0": [19, 90],
  "1": [23, 25, 55],
  "2": [],
  "3": [31, 73],
  "4": [31, 55, 91, 63]
}
```

**4. Indicate how many odd numbers are in each list in JSON format:**
```
{
  "0": 2,
  "1": 3,
  "2": 0,
  "3": 2,
  "4": 4
}
```

**5. Sort the lists in order of how many odd numbers they contain. Output a JSON list, each element is the index of the list, sorted in order of how many odd numbers they contain.**
```
[2, 0,

# More Advanced Prompting for sorting with more chain of thought
We can see the result is nearly perfect.But not yet perfect, so I add more instruction to make it better. The result, I can said it is perfect.

In [59]:
import json
# Write your code here.
def advanced_prompt(random_lists, model='gemini-pro', verbose=True):
    prompt = f"""I have some random lists of numbers: {random_lists}. 
    Please do the following using text only, don't output any code:
     1. Assign an index to each list in JSON format, key is index, value is the list.
     2. Identify each number in the list is odd or even in a JSON format, for example, if the input lists is [[1, 2, 3, 4, 5], [6, 7, 8, 9, 10]], the output should be {{ "0": {{"1": "odd", "2": "even", "3": "odd", "4": "even", "5": "odd"}}, "1": {{"6": "even", "7": "odd", "8": "even", "9": "odd", "10": "even"}} }}.
     3. grab all odd numbers are in each list in the json format, key is index, value is all odd numbers. Remember, odd numbers ends with 1, 3, 5, 7, 9. You should not include any even numbers that ends with 2, 4, 6, 8, 0. For example, 81, 73, 55, 37, 19 are odd number, while 82, 74, 56, 38, 20 are even numbers.
     4. Indicated how many odd numbers are in each list in the json format, key is index, value is how many odd numbers it includes.
     5. Sort the lists in order of how many odd numbers they contain. Output a json list, each element is the index of the list, sorted in order of how many odd numbers they contain.
     6. Sort the lists in order of how many odd numbers they contain. Output a json list, each element is the list input, sorted in order of how many odd numbers they contain."""
    
    model = genai.GenerativeModel('gemini-pro')
    response = model.generate_content(prompt)
    if verbose:
        print(f"INPUT: {random_lists}")
        print(f"OUTPUT: {response.text}")
        print()
    try:
        return json.loads(response.text.split("```")[-2].replace("json", ""))
    except:
        #print(response.text)
        return None
    


# Prepare the list useing random
import random
random_lists = [random.sample(range(1, 100), 5) for _ in range(5)]
result = advanced_prompt(random_lists)

INPUT: [[54, 68, 51, 24, 12], [63, 52, 87, 93, 33], [85, 30, 34, 21, 83], [84, 4, 46, 39, 33], [70, 61, 92, 10, 83]]
OUTPUT: 1. **Assigning an index to each list in JSON format:**
```json
{
  "0": [54, 68, 51, 24, 12],
  "1": [63, 52, 87, 93, 33],
  "2": [85, 30, 34, 21, 83],
  "3": [84, 4, 46, 39, 33],
  "4": [70, 61, 92, 10, 83]
}
```

2. **Identifying each number in the list as odd or even in JSON format:**
```json
{
  "0": {"54": "even", "68": "even", "51": "odd", "24": "even", "12": "even"},
  "1": {"63": "odd", "52": "even", "87": "odd", "93": "odd", "33": "odd"},
  "2": {"85": "odd", "30": "even", "34": "even", "21": "odd", "83": "odd"},
  "3": {"84": "even", "4": "even", "46": "even", "39": "odd", "33": "odd"},
  "4": {"70": "even", "61": "odd", "92": "even", "10": "even", "83": "odd"}
}
```

3. **Grabbing all odd numbers in each list in JSON format:**
```json
{
  "0": [51],
  "1": [63, 87, 93, 33],
  "2": [85, 21, 83, 39],
  "3": [33],
  "4": [61, 83]
}
```

4. **Indicating ho

## Evaluate it 

In [60]:
import json
import itertools

def process_lists(random_lists):
    indexed_lists = {str(i): lst for i, lst in enumerate(random_lists)}
    odd_even_map = {}
    odd_numbers = {}
    odd_counts = {}

    # Process each list
    for index, lst in indexed_lists.items():
        odd_even_map[index] = {num: 'odd' if num % 2 != 0 else 'even' for num in lst}
        odd_numbers[index] = [num for num in lst if num % 2 != 0]
        odd_counts[index] = len(odd_numbers[index])

    # Sorting by number of odd numbers and grouping lists with equal counts
    sorted_by_odd_count_indices = sorted(odd_counts, key=odd_counts.get)
    grouped_indices = {}
    for index in sorted_by_odd_count_indices:
        count = odd_counts[index]
        if count not in grouped_indices:
            grouped_indices[count] = []
        grouped_indices[count].append(index)

    # Generating permutations for groups with the same odd counts
    all_permutations = []
    for group in grouped_indices.values():
        if len(group) > 1:
            for permutation in itertools.permutations(group):
                permuted_list = []
                for idx in sorted_by_odd_count_indices:
                    if idx in group:
                        permuted_list.append(indexed_lists[permutation[group.index(idx)]])
                    else:
                        permuted_list.append(indexed_lists[idx])
                all_permutations.append(permuted_list)
        else:
            all_permutations.append([indexed_lists[idx] for idx in sorted_by_odd_count_indices])

    result = {
        "indexed_lists": indexed_lists,
        "odd_even_map": odd_even_map,
        "odd_numbers": odd_numbers,
        "odd_counts": odd_counts,
        "sorted_indices_by_odd_counts": sorted_by_odd_count_indices,
        "sorted_lists_by_odd_counts": all_permutations
    }

    # Optionally, convert result to JSON string with indentation
    # return json.dumps(result, indent=4)
    return all_permutations


In [69]:
# random_lists = [random.sample(range(1, 100), 5) for _ in range(5)]
# To make sure the order of the same odd number count didn't influcene the result, we enumerate all possible permutations of the same odd number count.
random_lists = [[1,3,5,7,9], [2,4,6,8,10], [1,2,3,4,5], [1,1,5,7,9], [1,5,5,7,9]]
process_lists(random_lists)

[[[2, 4, 6, 8, 10],
  [1, 2, 3, 4, 5],
  [1, 3, 5, 7, 9],
  [1, 1, 5, 7, 9],
  [1, 5, 5, 7, 9]],
 [[2, 4, 6, 8, 10],
  [1, 2, 3, 4, 5],
  [1, 3, 5, 7, 9],
  [1, 1, 5, 7, 9],
  [1, 5, 5, 7, 9]],
 [[2, 4, 6, 8, 10],
  [1, 2, 3, 4, 5],
  [1, 3, 5, 7, 9],
  [1, 1, 5, 7, 9],
  [1, 5, 5, 7, 9]],
 [[2, 4, 6, 8, 10],
  [1, 2, 3, 4, 5],
  [1, 3, 5, 7, 9],
  [1, 5, 5, 7, 9],
  [1, 1, 5, 7, 9]],
 [[2, 4, 6, 8, 10],
  [1, 2, 3, 4, 5],
  [1, 1, 5, 7, 9],
  [1, 3, 5, 7, 9],
  [1, 5, 5, 7, 9]],
 [[2, 4, 6, 8, 10],
  [1, 2, 3, 4, 5],
  [1, 1, 5, 7, 9],
  [1, 5, 5, 7, 9],
  [1, 3, 5, 7, 9]],
 [[2, 4, 6, 8, 10],
  [1, 2, 3, 4, 5],
  [1, 5, 5, 7, 9],
  [1, 3, 5, 7, 9],
  [1, 1, 5, 7, 9]],
 [[2, 4, 6, 8, 10],
  [1, 2, 3, 4, 5],
  [1, 5, 5, 7, 9],
  [1, 1, 5, 7, 9],
  [1, 3, 5, 7, 9]]]

In [70]:
import random
import tqdm
trial = 0
correct = 0

for i in tqdm.tqdm(range(100)):
    random_lists = [random.sample(range(1, 100), 5) for _ in range(5)]
    result = advanced_prompt(random_lists, verbose=False)
    isCorrect = 0
    for permuted_list in process_lists(random_lists):
        isCorrect = int(json.dumps(result) == json.dumps(permuted_list))
        if isCorrect:
            correct += isCorrect
            break
    trial += 1

100%|██████████| 100/100 [18:26<00:00, 11.06s/it]


In [71]:
print(f"Correct: {correct}/{trial} ({correct/trial:.2%})")

Correct: 33/100 (33.00%)


# Yet after evaluation, it is still 29% accuracy, not good enough.

# Generazlize to make it support even number, too

In [37]:
import json
# Write your code here.
def advanced_prompt_generalize(random_lists, model='gemini-pro', verbose=True, target = "odd"):
    prompt = f"""I have some random lists of numbers: {random_lists}. 
    Please do the following using text only, don't output any code:
     1. Assign an index to each list in JSON format, key is index, value is the list.
     2. Identify each number in the list is odd or even in a JSON format, for example, if the input lists is [[1, 2, 3, 4, 5], [6, 7, 8, 9, 10]], the output should be {{ "0": {{"1": "odd", "2": "even", "3": "odd", "4": "even", "5": "odd"}}, "1": {{"6": "even", "7": "odd", "8": "even", "9": "odd", "10": "even"}} }}.
     3. grab all {target} numbers are in each list in the json format, key is index, value is all {target} numbers. Remember, odd numbers ends with 1, 3, 5, 7, 9. You should not include any even numbers that ends with 2, 4, 6, 8, 0. For example, 81, 73, 55, 37, 19 are odd number, while 82, 74, 56, 38, 20 are even numbers.
     4. Indicated how many {target} numbers are in each list in the json format, key is index, value is how many {target} numbers it includes.
     5. Sort the lists in order of how many {target} numbers they contain. Output a json list, each element is the index of the list, sorted in order of how many {target} numbers they contain.
     6. Sort the lists in order of how many {target} numbers they contain. Output a json list, each element is the list input, sorted in order of how many {target} numbers they contain."""
    
    model = genai.GenerativeModel('gemini-pro')
    response = model.generate_content(prompt)
    if verbose:
        print(f"INPUT: {random_lists}")
        print(f"OUTPUT: {response.text}")
        print()
    return json.loads(response.text.split("```")[-2].replace("json", ""))
    


# Prepare the list useing random
import random
random_lists = [random.sample(range(1, 100), 5) for _ in range(5)]
result = advanced_prompt_generalize(random_lists, target="even")

INPUT: [[90, 63, 22, 57, 52], [21, 52, 63, 25, 33], [23, 39, 15, 7, 22], [21, 19, 58, 67, 12], [48, 64, 44, 57, 75]]
OUTPUT: **1. Assign an index to each list in JSON format**
```json
{
  "0": [90, 63, 22, 57, 52],
  "1": [21, 52, 63, 25, 33],
  "2": [23, 39, 15, 7, 22],
  "3": [21, 19, 58, 67, 12],
  "4": [48, 64, 44, 57, 75]
}
```

**2. Identify each number in the list is odd or even in a JSON format**

```json
{
  "0": {
    "90": "even",
    "63": "odd",
    "22": "even",
    "57": "odd",
    "52": "even"
  },
  "1": {
    "21": "odd",
    "52": "even",
    "63": "odd",
    "25": "odd",
    "33": "odd"
  },
  "2": {
    "23": "odd",
    "39": "odd",
    "15": "odd",
    "7": "odd",
    "22": "even"
  },
  "3": {
    "21": "odd",
    "19": "odd",
    "58": "even",
    "67": "odd",
    "12": "even"
  },
  "4": {
    "48": "even",
    "64": "even",
    "44": "even",
    "57": "odd",
    "75": "odd"
  }
}
```

**3. Grab all even numbers are in each list in the json format**

```json
{


# Writeup 1: Report & Reflection

Fill in the following three sections by reflecting on your results.

## Reflect on prompting effectiveness

Only the sorting can be manipulated by different prompts. I iterate it several times, and now it is super effective in sorting random lists according to the number of odd numbers it contains.

## Envision possible improvements


I have done my best job. Nevertheless, during the evaluation, there was still only a 33% accuracy rate, indicating there is definitely room for improvement. To enhance it, we can use more Chain of Thought to make Gemini work like a judge to assess the correctness of the output.

